In [ ]:
import pandas as pd

# Parâmetros gerais para a análise e processamento dos daddos
name_target = 'NU_NOTA_MT'
name_id = 'NU_INSCRICAO'

In [ ]:
# Análise dos dados de teste e de treino
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
print(f'Train shape: {df_train.shape}, Test shape: {df_test.shape}')

if name_target not in df_train.columns :
    print('ERROR: dataframe de treino não possui target')
    
elif df_test.columns.size >= (df_train.drop(columns=name_target).columns.size)\
        and all(elem in df_test for elem in df_train.drop(columns=name_target).columns) :
    print('ALERT: dataframes de teste e treino ok')

elif all(df_train[df_test.columns].columns == df_test.columns) :
    print('DONE: convertendo colunas dataframe de treino igual ao de teste + target')
    df_train = pd.concat([df_train[df_test.columns], df_train[name_target]], axis=1)
    print('DONE: gravando dataframe de treino formatado')
    df_train.to_csv('../data/train.csv', index=False)
else :
    print('ERROR: colunas dos Data frames de treino e testes são diferentes')

print(df_train.info())
del df_test, df_train

## Análise de Feature
df_train = pd.read_csv('../data/train.csv')
var = df_train['NU_IDADE'].copy()
import seaborn as sns
import statsmodels.api as sm
from IPython.core.pylabtools import figsize
%matplotlib inline
figsize(12, 8)
sns.set()
sns.heatmap(df_train.corr(), square=True);

sns.distplot(var)

sns.lineplot(var, df_train[name_target])

sns.scatterplot(var, df_train[name_target])

sm.qqplot(var, fit=True, line="45");

## Análise de dados nulos e zerados
df_train = pd.read_csv('../data/train.csv')
print(f'dados Nulos: {df_train[(df_train["TP_PRESENCA_LC"] == 0) | (df_train["TP_PRESENCA_LC"] == 2) ][name_target].unique()}')
print(f'dados Zerados: {df_train[(df_train["NU_NOTA_LC"] == 0)][name_target].unique()}')
del df_train

In [ ]:
# Parâmetros para remover alunos ausentes da prova de matemática
lin_null = [('TP_PRESENCA_LC', 0), ('TP_PRESENCA_LC', 2)]
lin_zero = [('NU_NOTA_LC', 0)]

# Parâmetro para remover alunos outliers restantes
lin_MT_0 = [('NU_NOTA_MT', 0)]

In [ ]:
# Classe para a leitura dos arquivos e carregar os dataframes
from data_source import DataSource
data = DataSource(name_id=name_id, 
                  name_target=name_target, 
                  rows_remove=(lin_null + lin_zero), 
                  outliers_remove=lin_MT_0)

In [ ]:
## Optional manual analise columns. 
# Syntax: COLUMN_NAME       TYPE    [FILLNA ENCODE  [DROP_FIRST]] COMMENTS
col_analise = [             
    ('NU_INSCRICAO',        None), # only unique values
    ('CO_UF_RESIDENCIA',    None), # duplicated
    ('SG_UF_RESIDENCIA',    'cat',  None,   False),
    ('NU_IDADE',            'num',  None,   False),
    ('TP_SEXO',             'cat',  None,   False),
    ('TP_COR_RACA',         'cat',  None,   True,   True),
    ('TP_NACIONALIDADE',    None), # insignificant correlation
    ('TP_ST_CONCLUSAO',     'cat',  None,   False),
    ('TP_ANO_CONCLUIU',     None), # missing values
    ('TP_ESCOLA',           'cat',  None,   True,   True),
    ('TP_ENSINO',           'cat',  None,   True,   False),
    ('IN_TREINEIRO',        'cat',  None,   False),	
    ('TP_DEPENDENCIA_ADM_ESC','cat',None,   True,   False),
    ('IN_BAIXA_VISAO',      None),	
    ('IN_CEGUEIRA',         None), 	
    ('IN_SURDEZ',           None), 	
    ('IN_DISLEXIA',         None), 	
    ('IN_DISCALCULIA',      None), 	
    ('IN_SABATISTA',        None), 	
    ('IN_GESTANTE',         None), 	
    ('IN_IDOSO',            None), 	
    ('TP_PRESENCA_CN',      None), 	
    ('TP_PRESENCA_CH',      None), 	
    ('TP_PRESENCA_LC',      None), 	
    ('CO_PROVA_CN',         None), 	
    ('CO_PROVA_CH',         None), 	
    ('CO_PROVA_LC',         None), 	
    ('CO_PROVA_MT',         None), 	
    ('NU_NOTA_CN',          'num',   459.8,  False),	
    ('NU_NOTA_CH',          'num',   532.0,  False),	
    ('NU_NOTA_LC',          'num',   None,   False),	
    ('NU_NOTA_REDACAO',     'num',   None,   False),	
    ('TP_LINGUA',           'cat',   None,   False),
    ('TP_STATUS_REDACAO',   None),	
    ('NU_NOTA_COMP1',       'num',   None,   False),	
    ('NU_NOTA_COMP2',       'num',   None,   False),
    ('NU_NOTA_COMP3',       'num',   None,   False),
    ('NU_NOTA_COMP4',       'num',   None,   False),
    ('NU_NOTA_COMP5',       'num',   None,   False),
    ('Q001',                'cat',   None,   False),	
    ('Q002',                'cat',   None,   False),
    ('Q006',                'cat',   None,   False),
    ('Q024',                'cat',   None,   False),
    ('Q025',                'cat',   None,   False),
    ('Q026',                'cat',   None,   False),
    ('Q027',                None), # missing values	
    ('Q047',                'cat',   None,   False),
    ]

In [ ]:
# Classe para o pré processamento das features
from preprocessing import Preprocessing
pre = Preprocessing(data, col_analise=col_analise, rfe='LR', pca=False)

In [ ]:
# Classe para definir e treinar o modelo
from model_training import ModelTraining
model = ModelTraining(pre, model_training='RF')
trained_model = model.training()

In [ ]:
# Classe para rodar o modelo e gerar o resultado
from model_inference import ModelInference
y_pred = ModelInference(trained_model).predict()

In [ ]:
# Pós tratamento para os alunos que não fizeram a prova:
df_removed = data.get_removed_rows(name_columns=[name_id], is_train_stage=False)
df_removed[name_target] = 0

# Gravação da resposta
df_answer = data.get_columns(name_columns=[name_id], is_train_stage=False)
df_answer[name_target] = y_pred.round(2)
df_answer = pd.concat([df_answer,df_removed])
df_answer.to_csv('../answer.csv', index=False)

In [ ]:
# histórico das submissões
# LinearRegression:
# 92.12%  StandardScaler->CatBoostEncoder->LinearRegression
# 93.24%  StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.57%  Seleção manual->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.58%  Seleção manual(2)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.61%  Seleção manual(3)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.60%  Seleção manual(3)->Seleção RFE(LinearRegression,50%)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.52%  Seleção manual(3)->Seleção RFE(LinearRegression,PCA(.95))->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# RandomForestRegressor:
# 93.57%  Seleção manual(3)->Seleção RFE(LinearRegression,PCA(.95))->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.56%  Seleção manual(3)->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.61%  Seleção manual(3)->Seleção RFE(LinearRegression,50%)->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.59%  Seleção manual(3)->Seleção RFE(DecisionTreeRegressor,50%)->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.55%  Seleção manual(3)->Seleção RFE(DecisionTreeRegressor,PCA(.95))->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.57%  Trat. Outliers->Seleção manual(3)->Seleção RFE(LinearRegression,50%)->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.60%  Trat. Outliers->Seleção manual(3)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.17%  Trat. Outliers->Seleção RFE(LinearRegression,50%)->StandardScaler->CatBoostEncoder->RandomForestRegressor->Pós Processamento
# 93.07%  Trat. Outliers->Seleção RFE(LinearRegression,50%)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento